Must needed setup before every assignment

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 56kB/s 
     |████████████████████████████████| 204kB 51.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=3e9e09547dad2cd8e7084c14529f12af3047890005a9dce7a945f105247550c4
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Imports

In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import *

Setup Spark

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Check if Spark is working

In [0]:
spark

Variables

In [0]:
#dataPath = 'graph-small.txt' # node # 1 ~ 100
dataPath = 'graph-full.txt' # node # 1 ~ 1000

# check min and max node #
mi = float('inf')
Mx = -float('inf')

Open File

In [0]:
with open(dataPath) as f:
    for line in f:
        a, b = map(int, line.split())
        mi = min(mi, a, b)
        Mx = max(Mx, a, b)

n = Mx

lines = sc.textFile(dataPath)
edges_uv = lines.map(lambda line: list(map(int, line.split())))
edges_vu = lines.map(lambda line: list(map(int, line.split()[::-1])))

Same definitions

In [0]:
def aggregateUV(st, v):
    st.add(v)
    return st

def aggregateUU(st1, st2):
    return st1 | st2

Get Edges

In [0]:
u_outNeigs = edges_uv.aggregateByKey(set(), aggregateUV, aggregateUU)
v_inNeigs  = edges_vu.aggregateByKey(set(), aggregateUV, aggregateUU)
# u_outDegs = u_outNeigs.map(lambda u_st: (u_st[0], len(u_st[1])))

M_uv = {}
M_vu = {}
for u, outNeigs in u_outNeigs.collect():
    M_uv[u] = outNeigs

for v, inNeigs in v_inNeigs.collect():
    M_vu[v] = inNeigs

u_outNeigs = sc.broadcast(M_uv)
v_inNeigs  = sc.broadcast(M_vu)


numIters = 40 #Number of iterations

Here we need additional definition

In [0]:
def LTh(h):
    LT = u_outNeigs.value
    new_a = [0] * n
    for j, hj in enumerate(h, 1): # 0/1 indexing
        if hj == 0:
            continue
        for i in LT[j]:
            new_a[i - 1] += hj # 0/1 indexing
    return new_a

def L_a(a):
    L  = v_inNeigs.value
    new_h = [0] * n
    for j, aj in enumerate(a, 1): # 0/1 indexing
        if aj == 0:
            continue
        for i in L[j]:
            new_h[i - 1] += aj # 0/1 indexing
    return new_h

Output

In [0]:
h = [1] * n


for _ in range(numIters):
    a = LTh(h)
    a = [v / max(a) for v in a]
    h = L_a(a)
    h = [v / max(h) for v in h]

h_id = zip(h, range(1, n + 1))
a_id = zip(a, range(1, n + 1))

h_id = sorted(h_id, reverse=True)
h_id_reverse = h_id[::-1]
a_id = sorted(a_id, reverse=True)
a_id_revese = a_id[::-1]

print(h_id[:5])
print(h_id_reverse[:5])
print(a_id[:5])
print(a_id_revese[:5])

[(1.0, 840), (0.9499618624906543, 155), (0.8986645288972263, 234), (0.8634171101843789, 389), (0.8632841092495218, 472)]
[(0.04206685489093652, 23), (0.057790593544330145, 835), (0.06453117646225177, 141), (0.0660265937341849, 539), (0.07678413939216452, 889)]
[(1.0, 893), (0.9635572849634398, 16), (0.9510158161074015, 799), (0.9246703586198443, 146), (0.899866197360405, 473)]
[(0.05608316377607618, 19), (0.06653910487622794, 135), (0.07544228624641901, 462), (0.08171239406816942, 24), (0.08571673456144875, 910)]
